# Maximum Likelihood Estimation (MLE)
- likelihood (가능도, 우도) 
- 최대 가능도 추정 / 최대 우도 추정

$K \sim B(n,\theta)  $ <br><br>
$ P(K = k) = \binom{n}{k}\theta^{k}(1-\theta)^{n-k} $ <br><br>
         $ = \frac{n!}{k!(n-k)!} \cdot\theta^{k}(1-\theta)^{n-k}$ <br><br>
- observation 을 가장 잘 나타내는 $\theta$ 를 찾아내는 과정

# Optimization via Gradient Descent

# Overfitting and Regurlarization

# Training and Test Dataset

# Learing Rate

# Data Processing

# 5주차 퀴즈 관련 풀이

In [3]:
import torch
torch_tensor = torch.FloatTensor([[[1],[2],[3]],[[4],[5],[6]], [[7],[8],[9]]])
print(torch_tensor.squeeze(), torch_tensor.squeeze().shape)

tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]]) torch.Size([3, 3])


In [4]:
torch_t1 = torch.FloatTensor([1,2,3])
torch_t2 = torch.FloatTensor([[4],[5],[6]])
print(torch_t1+torch_t2)

tensor([[5., 6., 7.],
        [6., 7., 8.],
        [7., 8., 9.]])


In [5]:
torch_tensor = torch.FloatTensor([[1,2,3,4],[5,6,7,8]])
torch_tensor = torch_tensor.view([-1,2])
print(torch_tensor)

tensor([[1., 2.],
        [3., 4.],
        [5., 6.],
        [7., 8.]])


In [7]:
torch_tensor1 = torch.FloatTensor([[1,2,3,4]])
torch_tensor2 = torch.FloatTensor([[5,6,7,8]])
print(torch.cat([torch_tensor1, torch_tensor2], axis = 0))

tensor([[1., 2., 3., 4.],
        [5., 6., 7., 8.]])
